In [1]:
from subprocess import check_output, check_call
from getpass import getpass
import nibabel as nib
import numpy as np
from glob import glob
import csv
import os
from subprocess import Popen, Popen, PIPE
import os
from glob import glob
from subprocess import Popen, PIPE
import json
from nipype.interfaces import fsl
from nipype.interfaces.fsl import RobustFOV, Reorient2Std
from nipype.interfaces.c3 import C3dAffineTool
import argparse
from getpass import getpass
import shutil
import pandas as pd
import csv

In [34]:
import re

In [2]:
base_dir = "/data/henry7/PBR/subjects/"

In [28]:
df = pd.read_csv("/home/sf522915/SKYRA_sienax_flairmixedmodel.csv")

In [111]:
writer = open("/home/sf522915/SKYRA_sienax_withscannerinfo_new.csv", "w")
spreadsheet = csv.DictWriter(writer, 
                             fieldnames=["msid", "Scan Date",
                                        "vscale", 
                                        "brain vol (u, mm3)",
                                        "WM vol (u, mm3)", 
                                        "GM vol (u, mm3)",
                                        "vCSF vol (u, mm3)",
                                        "cortical vol (u, mm3)",
                                        "lesion vol (u, mm3)",
                                         "mseID", "flair_file",
                                         'software', 't1_file', 
                                         "flipangle", "TR", "TE","InversionTime", "sDistortionCorrFilter", "ucTablePositioningMode"])
spreadsheet.writeheader()


for _, row in df.iterrows():
    mse = row['mseID']
    msid = row["msid"]
    t1 = row["t1_file"]
    flair = row["flair_file"]
    scandate = row["Scan Date"]
    vscale = row["vscale"]
    brainvol = row["brain vol (u, mm3)"]
    wm = row["WM vol (u, mm3)"]
    gm = row["GM vol (u, mm3)"]
    csf = row["vCSF vol (u, mm3)"]
    cort_vol = row["cortical vol (u, mm3)"]
    les = row["lesion vol (u, mm3)"]
    
    row = {"msid": msid, "mseID": mse, "t1_file": t1, "flair_file": flair, "Scan Date": scandate, \
          "vscale": vscale, "brain vol (u, mm3)": brainvol,"WM vol (u, mm3)": wm, "GM vol (u, mm3)": gm,\
          "vCSF vol (u, mm3)": csf, "cortical vol (u, mm3)": cort_vol, "lesion vol (u, mm3)": les}
    
    print(mse)
    dicom_path = "/working/henry_temp/PBR/dicoms/"+ mse
    if os.path.exists(dicom_path):
        series = t1.split("-")[2].lstrip("0")
        print(series)
        dicom = glob("/working/henry_temp/PBR/dicoms/{0}/E*/{1}/*.DCM".format(mse,series))[0]
        print(dicom)
        cmd = ["dcmdump", dicom]
        proc = Popen(cmd, stdout=PIPE)
        lines = str([l.decode("utf-8").split() for l in proc.stdout.readlines()[:]])
        
        proc = Popen("dcmdump {} | grep SoftwareVersion".format(dicom), stdout=PIPE, shell=True)
        proc.wait()
        res = proc.stdout.readlines()
        software = [r.decode("utf-8") for r in res][0][15:40]
        print(software, "software")
        row["software"] =  software

        proc = Popen("dcmdump {} | grep FlipAngle".format(dicom), stdout=PIPE, shell=True)
        proc.wait()
        res = proc.stdout.readlines()
        flipangle = [r.decode("utf-8") for r in res][0][15:21]
        print(flipangle, "flipangle")
        row["flipangle"] =  flipangle
        
        proc = Popen("dcmdump {} | grep RepetitionTime".format(dicom), stdout=PIPE, shell=True)
        proc.wait()
        res = proc.stdout.readlines()
        RepetitionTime = [r.decode("utf-8") for r in res][0][15:21]
        print(RepetitionTime, "TR")
        row["TR"] = RepetitionTime 
        
        proc = Popen("dcmdump {} | grep EchoTime".format(dicom), stdout=PIPE, shell=True)
        proc.wait()
        res = proc.stdout.readlines()
        EchoTime = [r.decode("utf-8") for r in res][0][15:21]
        print(EchoTime, "TE")
        row["TE"] = EchoTime
        
        try: 
            proc = Popen("dcmdump {} | grep InversionTime".format(dicom), stdout=PIPE, shell=True)
            proc.wait()
            res = proc.stdout.readlines()
            InversionTime = [r.decode("utf-8") for r in res][0][15:21]
            print(InversionTime, "InversionTime")
            row["InversionTime"] = InversionTime
        except Exception:
            pass
        
        
        try:
            proc = Popen("/netopt/afni/dicom_hdr -sexinfo {} | grep sDistortionCorrFilter.ucMode".format(dicom), stdout=PIPE, shell=True)
            proc.wait()
            res = proc.stdout.readlines()
            sDistortionCorrFilter = [r.decode("utf-8") for r in res][0]
            row["sDistortionCorrFilter"] = sDistortionCorrFilter
            print(sDistortionCorrFilter, "sDistortionCorrFilter")
    
        except Exception:
            pass
        
        
        try:
            proc = Popen("/netopt/afni/dicom_hdr -sexinfo {} | grep ucTablePositioningMode".format(dicom),\
                             stdout=PIPE, shell=True)
            proc.wait()
            res = proc.stdout.readlines()
            ucTablePositioningMode = ([r.decode("utf-8") for r in res])[0]
            row["ucTablePositioningMode"] = ucTablePositioningMode
            print(ucTablePositioningMode, "ucTablePositioningMode")
    
        except Exception:
            pass
         
    
        
        
        spreadsheet.writerow(row)

writer.close()
        
        
        
        
        
        
        
        

mse7538
5
/working/henry_temp/PBR/dicoms/mse7538/E2/5/E2S5I125.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse8959
5
/working/henry_temp/PBR/dicoms/mse8959/E2/5/E2S5I22.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse5779
5
/working/henry_temp/PBR/dicoms/mse5779/E1/5/E1S5I138.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse5838
5
/working/henry_temp/PBR/dicoms/mse5838/E1/5/E1S5I56.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	

[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse6729
13
/working/henry_temp/PBR/dicoms/mse6729/E2/13/E2S13I84.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	1
 ucTablePositioningMode
mse5825
5
/working/henry_temp/PBR/dicoms/mse5825/E2/5/E2S5I120.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse5738
5
/working/henry_temp/PBR/dicoms/mse5738/E1/5/E1S5I48.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse6725
5
/working/henry_temp/PBR/

[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse5823
5
/working/henry_temp/PBR/dicoms/mse5823/E2/5/E2S5I3.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse5732
5
/working/henry_temp/PBR/dicoms/mse5732/E2/5/E2S5I134.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse4216
5
/working/henry_temp/PBR/dicoms/mse4216/E2/5/E2S5I11.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	1
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePo

[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse5092
3
/working/henry_temp/PBR/dicoms/mse5092/E2/3/E2S3I172.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	2
 ucTablePositioningMode
mse5754
6
/working/henry_temp/PBR/dicoms/mse5754/E2/6/E2S6I2.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse5758
4
/working/henry_temp/PBR/dicoms/mse5758/E2/4/E2S4I96.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePo

[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse4444
3
/working/henry_temp/PBR/dicoms/mse4444/E2/3/E2S3I165.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse4461
3
/working/henry_temp/PBR/dicoms/mse4461/E2/3/E2S3I148.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse4285
3
/working/henry_temp/PBR/dicoms/mse4285/E2/3/E2S3I134.DCM
[syngo MR D13]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse4702
3
/working

...

[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse7845
46
/working/henry_temp/PBR/dicoms/mse7845/E11457591/46/E11457591S46I60.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse7844
46
/working/henry_temp/PBR/dicoms/mse7844/E11457566/46/E11457566S46I72.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse7849
46
/working/henry_temp/PBR/dicoms/mse7849/E11459895/46/E11459895S46I172.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode

[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8144
57
/working/henry_temp/PBR/dicoms/mse8144/E11510188/57/E11510188S57I62.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8140
59
/working/henry_temp/PBR/dicoms/mse8140/E11507830/59/E11507830S59I10.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8151
40
/working/henry_temp/PBR/dicoms/mse8151/E11512208/40/E11512208S40I33.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	

[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8606
36
/working/henry_temp/PBR/dicoms/mse8606/E11584703/36/E11584703S36I21.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8609
47
/working/henry_temp/PBR/dicoms/mse8609/E11584814/47/E11584814S47I160.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8622
41
/working/henry_temp/PBR/dicoms/mse8622/E11592566/41/E11592566S41I71.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePo

[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8831
41
/working/henry_temp/PBR/dicoms/mse8831/E11650221/41/E11650221S41I119.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8848
7
/working/henry_temp/PBR/dicoms/mse8848/E11655638/7/E11655638S7I158.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse8850
53
/working/henry_temp/PBR/dicoms/mse8850/E11662485/53/E11662485S53I20.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePosi

[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse9100
46
/working/henry_temp/PBR/dicoms/mse9100/E11724502/46/E11724502S46I133.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse9272
47
/working/henry_temp/PBR/dicoms/mse9272/E11746375/47/E11746375S47I122.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistortionCorrFilter
ucTablePositioningMode	 = 	4
 ucTablePositioningMode
mse9279
43
/working/henry_temp/PBR/dicoms/mse9279/E11746579/43/E11746579S43I136.DCM
[syngo MR E11]            software
[9]    flipangle
[2300] TR
[2.98] TE
[900]  InversionTime
sDistortionCorrFilter.ucMode	 = 	2
 sDistor

In [ ]:
for